In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from sklearn.svm import SVR
from sklearn import svm
import seaborn as sns

In [41]:
df = pd.read_csv(r'C:\Users\greym\Desktop\dataset for 446\energydata.csv')
df.head()

,REF_DATE,GEO,DGUID,Type of dwelling,Energy type,Energy consumption,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,2011,Canada,2016A000011124,Single-detached,"Total, all energy types",Gigajoules,Gigajoules,143,units,0,v101904406,1.1.1.1,944421514.0,NaN,NaN,NaN,0
1,2011,Canada,2016A000011124,Single-detached,"Total, all energy types",Gigajoules per household,Gigajoules,143,units,0,v101904407,1.1.1.2,115.3,NaN,NaN,NaN,1
2,2011,Canada,2016A000011124,Single-detached,"Total, all energy types",Number of households,Number,223,units,0,v101904408,1.1.1.4,8191007.0,NaN,NaN,NaN,0
3,2011,Canada,2016A000011124,Single-detached,Electricity,Gigajoules,Gigajoules,143,units,0,v101904409,1.1.2.1,396962347.0,NaN,NaN,NaN,0
4,2011,Canada,2016A000011124,Single-detached,Electricity,Gigajoules per household,Gigajoules,143,units,0,v101904410,1.1.2.2,48.5,NaN,NaN,NaN,1


In [42]:
# drop uncessary data
df.drop(['DGUID','UOM_ID','SCALAR_ID','Type of dwelling','SCALAR_FACTOR','VECTOR','COORDINATE','STATUS','SYMBOL', 'DECIMALS','TERMINATED'],inplace=True, axis=1)
df.head()

,REF_DATE,GEO,Energy type,Energy consumption,UOM,VALUE
0,2011,Canada,"Total, all energy types",Gigajoules,Gigajoules,944421514.0
1,2011,Canada,"Total, all energy types",Gigajoules per household,Gigajoules,115.3
2,2011,Canada,"Total, all energy types",Number of households,Number,8191007.0
3,2011,Canada,Electricity,Gigajoules,Gigajoules,396962347.0
4,2011,Canada,Electricity,Gigajoules per household,Gigajoules,48.5


In [43]:
#drop null data
newdf = df.dropna()
newdf.head()

,REF_DATE,GEO,Energy type,Energy consumption,UOM,VALUE
0,2011,Canada,"Total, all energy types",Gigajoules,Gigajoules,944421514.0
1,2011,Canada,"Total, all energy types",Gigajoules per household,Gigajoules,115.3
2,2011,Canada,"Total, all energy types",Number of households,Number,8191007.0
3,2011,Canada,Electricity,Gigajoules,Gigajoules,396962347.0
4,2011,Canada,Electricity,Gigajoules per household,Gigajoules,48.5


In [44]:
# drop all data that does not have the energy type of electricity and has a location of canada
newdf.drop(newdf[newdf['GEO'] == 'Canada'].index, inplace = True)
newdf.drop(newdf[newdf['Energy type'] != 'Electricity'].index, inplace = True)
newdf.head()

C:\Users\greym\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,REF_DATE,GEO,Energy type,Energy consumption,UOM,VALUE
108,2011,Newfoundland and Labrador,Electricity,Gigajoules,Gigajoules,11934482.0
109,2011,Newfoundland and Labrador,Electricity,Gigajoules per household,Gigajoules,72.7
110,2011,Newfoundland and Labrador,Electricity,Proportion of total energy,Percent,77.6
111,2011,Newfoundland and Labrador,Electricity,Number of households,Number,164067.0
213,2011,Prince Edward Island,Electricity,Gigajoules,Gigajoules,1147772.0


In [45]:
# Select columns of electricity consumption that is gigajoules
energy=newdf.loc[newdf['Energy consumption'] == 'Gigajoules']
y = energy.iloc[:, [5]]
y.head()

,VALUE
108,11934482.0
213,1147772.0
318,9704911.0
363,685874.0
378,1429845.0


In [46]:
# Select column that is giga per household
household=newdf.loc[newdf['Energy consumption'] == 'Gigajoules per household']
z = household.iloc[:, [5]]
z.head()

,VALUE
109,72.7
214,26.5
319,36.0
364,31.0
379,23.6


In [47]:
# Select columns of proportion of total energy that is gigajoules
proportion=newdf.loc[newdf['Energy consumption'] == 'Proportion of total energy']
x2 = proportion.iloc[:, [5]]
x2.head()

,VALUE
110,77.6
215,28.6
320,47.9
365,56.2
380,35.1


In [48]:
# Select columns of proportion of number of households that is gigajoules
number=newdf.loc[newdf['Energy consumption'] == 'Number of households']
x3 = number.iloc[:, [5]]
x3.head()


,VALUE
111,164067.0
216,43300.0
321,269296.0
366,22136.0
381,60523.0


In [49]:
#drop the 12 extra rows 
perhousehold = z.drop([3349,3304,3109,2584,2299,1969,1129,1084,1039,814,2149,4039])
proportion = x2.drop([815,1040,1085,2150,2300,2585,3110,3305])
number= x3.drop([1086,1971,2301,2586,3111,3351])



In [50]:
#Inserting all 3 dataframes
perhousehold = perhousehold.reset_index(drop = True)
proportion = proportion.reset_index(drop =True)
number = number.reset_index(drop = True)
X = pd.concat([perhousehold,proportion,number],axis=1,keys=['Perhousehold', 'proportion', 'Number'])
X

,Perhousehold,proportion,Number
,VALUE,VALUE,VALUE
0,72.7,77.6,164067.0
1,26.5,28.6,43300.0
2,36.0,47.9,269296.0
3,31.0,56.2,22136.0
4,23.6,35.1,60523.0
...,...,...,...
118,31.7,43.7,141375.0
119,38.8,41.3,54159.0
120,16.3,43.1,347275.0


In [91]:
# dependent and independent variables
X1 = [X]
Y = y.reset_index(drop = True)
Y

,VALUE
0,11934482.0
1,1147772.0
2,9704911.0
3,685874.0
4,1429845.0
...,...
118,4480597.0
119,2099854.0
120,5651427.0
121,2979272.0


In [52]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=0)

In [114]:
sc_x = StandardScaler()
sc_y = StandardScaler()
x_I = sc_x.fit_transform(x_train)
y_p = sc_y.fit_transform(y_train)
print("Scaled X_l:")
print(x_I)
print("Scaled y_p:")
print(y_p)

Scaled X_l:
[[ 2.99559874e-01 -5.69402642e-01 -6.01537663e-01]
 [-9.77038608e-01 -6.19373699e-01 -2.23709484e-01]
 [-1.81456306e-02 -8.80760764e-01  4.86978198e+00]
 [ 1.55148281e-01  1.49411788e-01 -5.95854558e-01]
 [-4.05168700e-01  1.22504296e-01 -6.39351297e-01]
 [ 1.49371817e-01 -1.07295714e+00  1.02855876e+00]
 [-3.87839309e-01  1.52938174e+00 -5.77215259e-01]
 [-4.91815655e-01 -9.30731821e-01 -9.43857573e-02]
 [-7.51756523e-01  3.72359579e-01  7.84402654e-01]
 [ 1.48373494e+00 -2.31137028e-01 -5.94848765e-01]
 [-7.40203595e-01 -8.30789708e-01  6.72121832e-02]
 [ 1.57038189e+00 -1.15819205e-01 -5.57705139e-02]
 [ 1.20489499e-01  1.77539309e+00 -6.15649006e-01]
 [-1.25430887e+00 -3.81050198e-01 -2.46476675e-02]
 [ 4.09312685e-01 -5.57870860e-01  1.29029354e+00]
 [-7.74862378e-01  5.22272749e-01  4.69515904e-01]
 [ 1.37398213e+00  1.28721431e+00 -1.35791507e-01]
 [-2.14545397e-01  1.36793679e+00  8.61852449e-01]
 [ 1.84030600e-01 -4.19489472e-01 -1.51621394e-01]
 [ 2.53348164e-01 -

In [120]:
regressor = SVR(kernel = 'rbf')
regressor.fit(x_I, y_p)
y_pred = sc_y.inverse_transform(regressor.predict(sc_x.transform(x_test)))
y_pred

C:\Users\greym\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


array([ 4683353.15226073, 44410086.21602636,  9670046.39545558,
        7828518.65565601,  5359079.52452504, 12197677.40637489,
        2319112.33421287, 59939283.339112  , 10764198.01834247,
        2363684.77552248,  2736191.44601734, 26249338.78638487,
        2170241.27088135, 61214899.51130353,  7433495.31002143,
        3861109.50991147, 13460309.86917815,  3562701.48295285,
        3120651.38816878, 13704408.97629051, 15620511.10719279,
        9941884.80926106,  4312666.74782333,  2893009.41258401,
        5346815.01801904, 32621406.49985781, 84073533.27250047,
        3064663.43195793, 84273773.59661974,  4752893.85063831,
       43938490.35061692])

In [116]:
r2=r2_score(y_test,y_pred) # calculate R2
mae = mean_absolute_error(y_test, y_pred)# calculate MAE
mse = mean_squared_error(y_test, y_pred)# calculate MSE

print("MAE:", mae)
print("MSE:", mse)
print("R2:", r2)

MAE: 10961276.052105483
MSE: 581733323034376.6
R2: 0.6632517051173232
